# DATA LOAD

Read the original Excel information and transform it to be processed.

The original information is the folder 'data/original', classified by years and in some file by year

In every file exists multiple sheet, some of them with userful information and others that want be used.

The files and sheets will be detected using the cell headers. 

The preprocessed information will be stored in folder 'data/preprocessed', classified by language.


In [25]:
import json
import os.path
import sys
import csv
import numpy as np
import pandas as pd
from colorama import Fore

In [3]:
import spacy
from spacy_langdetect import LanguageDetector

In [4]:
pathori = "../data/original/"
pathdest = "../data/preprocessed/"

In [40]:
def validateFile(file, sheet, data):
    
    cols = data.columns
#    print (file + "/" + sheet + ": ")
    if (cols.size > 5):
        if (('Codi pregunta' in cols) or ('Comentari' in cols) or ('Tipus incidencia' in cols)):
            if (('Codi pregunta' in cols) and ('Comentari' in cols) and ('Tipus incidencia' in cols)):
                print (Fore.GREEN + file + "/" + sheet + ": VALID")
                return True
            else:
                print (Fore.RED + file + "/" + sheet + ": WARNING (see column names)")
                print (cols)
                return False
        else:
            print (Fore.LIGHTWHITE_EX + file + "/" + sheet + ": NO VALID")
            return False
    else:
        print (Fore.LIGHTWHITE_EX + file + "/" + sheet + ": NO VALID")
        return False


In [44]:
for folder in os.listdir(pathori):
    if os.path.isdir(pathori+folder):
        print (Fore.BLACK + 'Folder: ' + folder)
        for file in os.listdir(pathori+folder):
            if os.path.isfile(pathori+folder+"/"+file):
#                print ("    " + file)
                xlsx = pd.ExcelFile(pathori+folder+"/"+file)
                for sheet in xlsx.sheet_names:
#                    print ("    " + file + "/" + sheet)
                    data = pd.read_excel(xlsx, sheet)
#                    data.head()
#                    print(data.info)
#                    print(data.dtypes)
                    if validateFile(file, sheet, data):
                        print("Valid file")
                    

Folder: 17-18
assignatura_professor_comentaris_LS_17-18_1s.xlsx/Hoja1: VALID
Valid file
assignatura_professor_comentaris_LS_17-18_1s.xlsx/Notas: NO VALID
assignatura_professor_comentaris_LS_17-18_1s_M110.xlsx/Sheet0: VALID
Valid file
assignatura_professor_comentaris_LS_17-18_1s_M110.xlsx/Notas: NO VALID
assignatura_professor_comentaris_LS_17-18_2s.xlsx/Sheet0: VALID
Valid file
assignatura_professor_comentaris_LS_17-18_2s.xlsx/Notas: NO VALID
Folder: 18-19


XLRDError: Unsupported format, or corrupt file: Expected BOF record; found b',david,d'

In [31]:
print(Fore.BLACK + 'BLACK')
print(Fore.BLUE + 'BLUE')
print(Fore.CYAN + 'CYAN')
print(Fore.GREEN + 'GREEN')
print(Fore.LIGHTBLACK_EX + 'LIGHTBLACK_EX')
print(Fore.LIGHTBLUE_EX + 'LIGHTBLUE_EX')
print(Fore.LIGHTCYAN_EX + 'LIGHTCYAN_EX')
print(Fore.LIGHTGREEN_EX + 'LIGHTGREEN_EX')
print(Fore.LIGHTMAGENTA_EX + 'LIGHTMAGENTA_EX')
print(Fore.LIGHTRED_EX + 'LIGHTRED_EX')
print(Fore.LIGHTWHITE_EX + 'LIGHTWHITE_EX')
print(Fore.LIGHTYELLOW_EX + 'LIGHTYELLOW_EX')
print(Fore.MAGENTA + 'MAGENTA')
print(Fore.RED + 'RED')
print(Fore.RESET + 'RESET')
print(Fore.WHITE + 'WHITE')
print(Fore.YELLOW + 'YELLOW')

BLACK
BLUE
CYAN
GREEN
LIGHTBLACK_EX
LIGHTBLUE_EX
LIGHTCYAN_EX
LIGHTGREEN_EX
LIGHTMAGENTA_EX
LIGHTRED_EX
LIGHTWHITE_EX
LIGHTYELLOW_EX
MAGENTA
RED
RESET
WHITE
YELLOW
